In [1]:
import os
import sys
import numpy as np
import h5py
root = os.path.dirname(os.path.abspath(os.curdir))
sys.path.append(root)

import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.utils.data import Subset

from predify.utils.training import train_pcoders, eval_pcoders

from networks_2022 import BranchedNetwork
from data.ReconstructionTrainingDataset import CleanSoundsDataset
from data.ReconstructionTrainingDataset import NoisySoundsDataset

# Specify Network to train
TODO: This should be converted to a script that accepts arguments for which network to train

In [ ]:
pnet_name = 'pnet'
_train_datafile = 'clean_reconstruction_training_set'
SoundsDataset = CleanSoundsDataset
dset_kwargs = {}

In [2]:
snr = '3'
pnet_name = 'pnet_snr3'
_train_datafile = 'hyperparameter_pooled_training_dataset_random_order_noNulls'
SoundsDataset = NoisySoundsDataset
dset_kwargs = {'snr': snr}

In [3]:
from pbranchednetwork_all import PBranchedNetwork_AllSeparateHP
PNetClass = PBranchedNetwork_AllSeparateHP

# Parameters

In [4]:
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
print(f'Device: {DEVICE}')
BATCH_SIZE = 50
NUM_WORKERS = 2
PIN_MEMORY = True
NUM_EPOCHS = 70

lr = 1E-5
engram_dir = '/mnt/smb/locker/abbott-locker/hcnn/'
checkpoints_dir = f'{engram_dir}checkpoints/'
tensorboard_dir = f'{engram_dir}tensorboard/'
train_datafile = f'{engram_dir}{_train_datafile}.hdf5'

Device: cuda:0


In [5]:
!nvidia-smi

Fri Sep  9 16:24:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 515.48.07    Driver Version: 515.48.07    CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0 Off |                  N/A |
|  0%   27C    P8     8W / 250W |      4MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load network and optimizer

In [6]:
net = BranchedNetwork()
net.load_state_dict(torch.load(f'{engram_dir}networks_2022_weights.pt'))

/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:78: UserWarning: Inconsistent tf pad calculation in ConvLayer.
  warnings.warn('Inconsistent tf pad calculation in ConvLayer.')
/share/ctn/users/cf2794/Code/hallucnn/src/models/layers.py:173: UserWarning: Inconsistent tf pad calculation: 0, 1
  warnings.warn(f'Inconsistent tf pad calculation: {pad_left}, {pad_right}')


<All keys matched successfully>

In [7]:
pnet = PNetClass(net, build_graph=True)

In [8]:
pnet.eval()

PBranchedNetwork_AllSeparateHP(
  (backbone): BranchedNetwork(
    (speech_branch): Sequential(
      (conv1): ConvLayer(
        (block): Sequential(
          (0): Conv2d(1, 96, kernel_size=(6, 14), stride=(3, 3), padding=(2, 6))
          (1): ReLU()
        )
      )
      (rnorm1): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool1): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv2): ConvLayer(
        (block): Sequential(
          (0): Conv2d(96, 256, kernel_size=(5, 5), stride=(2, 2), padding=(1, 2))
          (1): ReLU()
        )
      )
      (rnorm2): LRNorm(
        (block): LocalResponseNorm(5, alpha=0.005, beta=0.75, k=1.0)
      )
      (pool2): PoolLayer(
        (block): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
      )
      (conv3): ConvLayer(
        (block): Sequential(
          (0): Conv2d(256, 512, kernel_size=

In [9]:
pnet.to(DEVICE)
optimizer = torch.optim.Adam(
    [{'params':getattr(pnet,f"pcoder{x+1}").pmodule.parameters(), 'lr':lr} for x in range(pnet.number_of_pcoders)],
    weight_decay=5e-4)

# Set up train/test dataset

In [10]:
train_dataset = SoundsDataset(
    train_datafile, subset=.9, **dset_kwargs)
test_dataset = SoundsDataset(
    train_datafile, subset=.9,
    train = False, **dset_kwargs
    )

In [11]:
train_loader = DataLoader(
    train_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )
eval_loader = DataLoader(
    test_dataset, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=NUM_WORKERS, pin_memory=PIN_MEMORY
    )

# Set up checkpoints and tensorboards

In [12]:
checkpoint_path = os.path.join(checkpoints_dir, f"{pnet_name}")
if not os.path.exists(checkpoint_path):
    os.makedirs(checkpoint_path)
checkpoint_path = os.path.join(checkpoint_path, pnet_name + '-{epoch}-{type}.pth')

# summarywriter
from torch.utils.tensorboard import SummaryWriter
tensorboard_path = os.path.join(tensorboard_dir, f"{pnet_name}")
if not os.path.exists(tensorboard_path):
    os.makedirs(tensorboard_path)
sumwriter = SummaryWriter(tensorboard_path, filename_suffix=f'')

# Train

In [ ]:
loss_function = torch.nn.MSELoss()
for epoch in range(1, NUM_EPOCHS+1):
    train_pcoders(pnet, optimizer, loss_function, epoch, train_loader, DEVICE, sumwriter)
    eval_pcoders(pnet, loss_function, epoch, eval_loader, DEVICE, sumwriter)

    # save checkpoints every 5 epochs
    if epoch % 5 == 0:
        torch.save(pnet.state_dict(), checkpoint_path.format(epoch=epoch, type='regular'))

Training Epoch: 1 [50/13482]	Loss: 35514.4609
Training Epoch: 1 [100/13482]	Loss: 33856.3945
Training Epoch: 1 [150/13482]	Loss: 32072.6426
Training Epoch: 1 [200/13482]	Loss: 30351.8027
Training Epoch: 1 [250/13482]	Loss: 27271.6016
Training Epoch: 1 [300/13482]	Loss: 26212.2793
Training Epoch: 1 [350/13482]	Loss: 22904.4570
Training Epoch: 1 [400/13482]	Loss: 22204.2480
Training Epoch: 1 [450/13482]	Loss: 21392.3672
Training Epoch: 1 [500/13482]	Loss: 19045.0059
Training Epoch: 1 [550/13482]	Loss: 17623.0371
Training Epoch: 1 [600/13482]	Loss: 16369.6934
Training Epoch: 1 [650/13482]	Loss: 16115.6201
Training Epoch: 1 [700/13482]	Loss: 14240.6357
Training Epoch: 1 [750/13482]	Loss: 13777.3047
Training Epoch: 1 [800/13482]	Loss: 12901.0420
Training Epoch: 1 [850/13482]	Loss: 12825.3184
Training Epoch: 1 [900/13482]	Loss: 11872.5615
Training Epoch: 1 [950/13482]	Loss: 11763.4502
Training Epoch: 1 [1000/13482]	Loss: 11647.2930
Training Epoch: 1 [1050/13482]	Loss: 11793.6016
Training Epo

Training Epoch: 1 [8750/13482]	Loss: 6655.8267
Training Epoch: 1 [8800/13482]	Loss: 6865.2637
Training Epoch: 1 [8850/13482]	Loss: 6544.4727
Training Epoch: 1 [8900/13482]	Loss: 6970.1553
Training Epoch: 1 [8950/13482]	Loss: 6812.4385
Training Epoch: 1 [9000/13482]	Loss: 6214.8384
Training Epoch: 1 [9050/13482]	Loss: 6737.5078
Training Epoch: 1 [9100/13482]	Loss: 6338.9316
Training Epoch: 1 [9150/13482]	Loss: 6588.7588
Training Epoch: 1 [9200/13482]	Loss: 6802.5410
Training Epoch: 1 [9250/13482]	Loss: 6514.0654
Training Epoch: 1 [9300/13482]	Loss: 6610.1338
Training Epoch: 1 [9350/13482]	Loss: 6629.9448
Training Epoch: 1 [9400/13482]	Loss: 6396.2954
Training Epoch: 1 [9450/13482]	Loss: 6822.5708
Training Epoch: 1 [9500/13482]	Loss: 6459.7798
Training Epoch: 1 [9550/13482]	Loss: 6625.3062
Training Epoch: 1 [9600/13482]	Loss: 6703.5132
Training Epoch: 1 [9650/13482]	Loss: 6577.0278
Training Epoch: 1 [9700/13482]	Loss: 6533.3340
Training Epoch: 1 [9750/13482]	Loss: 6395.3457
Training Epoc

Training Epoch: 2 [3900/13482]	Loss: 4521.2158
Training Epoch: 2 [3950/13482]	Loss: 4411.3242
Training Epoch: 2 [4000/13482]	Loss: 4321.6499
Training Epoch: 2 [4050/13482]	Loss: 4349.6797
Training Epoch: 2 [4100/13482]	Loss: 4174.0342
Training Epoch: 2 [4150/13482]	Loss: 4254.9570
Training Epoch: 2 [4200/13482]	Loss: 4342.5776
Training Epoch: 2 [4250/13482]	Loss: 4499.0830
Training Epoch: 2 [4300/13482]	Loss: 4384.2261
Training Epoch: 2 [4350/13482]	Loss: 4330.2275
Training Epoch: 2 [4400/13482]	Loss: 4354.7412
Training Epoch: 2 [4450/13482]	Loss: 4184.8809
Training Epoch: 2 [4500/13482]	Loss: 4147.5303
Training Epoch: 2 [4550/13482]	Loss: 4009.1179
Training Epoch: 2 [4600/13482]	Loss: 4016.0684
Training Epoch: 2 [4650/13482]	Loss: 4291.1372
Training Epoch: 2 [4700/13482]	Loss: 4140.5488
Training Epoch: 2 [4750/13482]	Loss: 4340.7466
Training Epoch: 2 [4800/13482]	Loss: 3970.8330
Training Epoch: 2 [4850/13482]	Loss: 3922.4495
Training Epoch: 2 [4900/13482]	Loss: 4205.3379
Training Epoc

Training Epoch: 2 [12600/13482]	Loss: 3219.5972
Training Epoch: 2 [12650/13482]	Loss: 3149.7546
Training Epoch: 2 [12700/13482]	Loss: 3181.9995
Training Epoch: 2 [12750/13482]	Loss: 3214.3687
Training Epoch: 2 [12800/13482]	Loss: 3298.6218
Training Epoch: 2 [12850/13482]	Loss: 3166.3269
Training Epoch: 2 [12900/13482]	Loss: 3310.7627
Training Epoch: 2 [12950/13482]	Loss: 3314.7268
Training Epoch: 2 [13000/13482]	Loss: 3145.5649
Training Epoch: 2 [13050/13482]	Loss: 3180.5891
Training Epoch: 2 [13100/13482]	Loss: 3173.1042
Training Epoch: 2 [13150/13482]	Loss: 3121.3225
Training Epoch: 2 [13200/13482]	Loss: 3215.6448
Training Epoch: 2 [13250/13482]	Loss: 3370.7686
Training Epoch: 2 [13300/13482]	Loss: 3256.5464
Training Epoch: 2 [13350/13482]	Loss: 3276.1116
Training Epoch: 2 [13400/13482]	Loss: 3341.3123
Training Epoch: 2 [13450/13482]	Loss: 3272.2568
Training Epoch: 2 [13482/13482]	Loss: 3237.9854
Training Epoch: 2 [1497/1497]	Loss: 3173.1875
Training Epoch: 3 [50/13482]	Loss: 3134.38

Training Epoch: 3 [7800/13482]	Loss: 2762.3594
Training Epoch: 3 [7850/13482]	Loss: 2732.4092
Training Epoch: 3 [7900/13482]	Loss: 2751.5593
Training Epoch: 3 [7950/13482]	Loss: 2752.1106
Training Epoch: 3 [8000/13482]	Loss: 2606.4763
Training Epoch: 3 [8050/13482]	Loss: 2730.6392
Training Epoch: 3 [8100/13482]	Loss: 2711.8989
Training Epoch: 3 [8150/13482]	Loss: 2768.2273
Training Epoch: 3 [8200/13482]	Loss: 2640.6934
Training Epoch: 3 [8250/13482]	Loss: 2603.0269
Training Epoch: 3 [8300/13482]	Loss: 2703.3542
Training Epoch: 3 [8350/13482]	Loss: 2672.4053
Training Epoch: 3 [8400/13482]	Loss: 2740.9822
Training Epoch: 3 [8450/13482]	Loss: 2675.0593
Training Epoch: 3 [8500/13482]	Loss: 2656.1238
Training Epoch: 3 [8550/13482]	Loss: 2597.9863
Training Epoch: 3 [8600/13482]	Loss: 2637.7417
Training Epoch: 3 [8650/13482]	Loss: 2726.5886
Training Epoch: 3 [8700/13482]	Loss: 2624.5857
Training Epoch: 3 [8750/13482]	Loss: 2630.8455
Training Epoch: 3 [8800/13482]	Loss: 2731.5029
Training Epoc

...

Training Epoch: 13 [10550/13482]	Loss: 1532.0734
Training Epoch: 13 [10600/13482]	Loss: 1513.2908
Training Epoch: 13 [10650/13482]	Loss: 1485.7692
Training Epoch: 13 [10700/13482]	Loss: 1501.2776
Training Epoch: 13 [10750/13482]	Loss: 1483.6371
Training Epoch: 13 [10800/13482]	Loss: 1485.2908
Training Epoch: 13 [10850/13482]	Loss: 1507.2706
Training Epoch: 13 [10900/13482]	Loss: 1478.3071
Training Epoch: 13 [10950/13482]	Loss: 1484.9973
Training Epoch: 13 [11000/13482]	Loss: 1506.0767
Training Epoch: 13 [11050/13482]	Loss: 1514.3499
Training Epoch: 13 [11100/13482]	Loss: 1514.7135
Training Epoch: 13 [11150/13482]	Loss: 1504.8612
Training Epoch: 13 [11200/13482]	Loss: 1475.3760
Training Epoch: 13 [11250/13482]	Loss: 1538.8278
Training Epoch: 13 [11300/13482]	Loss: 1509.1516
Training Epoch: 13 [11350/13482]	Loss: 1563.6666
Training Epoch: 13 [11400/13482]	Loss: 1513.9974
Training Epoch: 13 [11450/13482]	Loss: 1484.2275
Training Epoch: 13 [11500/13482]	Loss: 1506.2930
Training Epoch: 13 [

Training Epoch: 14 [5500/13482]	Loss: 1510.0514
Training Epoch: 14 [5550/13482]	Loss: 1494.7201
Training Epoch: 14 [5600/13482]	Loss: 1471.5193
Training Epoch: 14 [5650/13482]	Loss: 1462.5646
Training Epoch: 14 [5700/13482]	Loss: 1484.2953
Training Epoch: 14 [5750/13482]	Loss: 1515.9486
Training Epoch: 14 [5800/13482]	Loss: 1524.1588
Training Epoch: 14 [5850/13482]	Loss: 1567.0033
Training Epoch: 14 [5900/13482]	Loss: 1480.7257
Training Epoch: 14 [5950/13482]	Loss: 1442.6595
Training Epoch: 14 [6000/13482]	Loss: 1477.1831
Training Epoch: 14 [6050/13482]	Loss: 1491.3770
Training Epoch: 14 [6100/13482]	Loss: 1522.6396
Training Epoch: 14 [6150/13482]	Loss: 1523.9301
Training Epoch: 14 [6200/13482]	Loss: 1547.1469
Training Epoch: 14 [6250/13482]	Loss: 1446.4478
Training Epoch: 14 [6300/13482]	Loss: 1456.6877
Training Epoch: 14 [6350/13482]	Loss: 1454.3431
Training Epoch: 14 [6400/13482]	Loss: 1525.1084
Training Epoch: 14 [6450/13482]	Loss: 1494.5992
Training Epoch: 14 [6500/13482]	Loss: 14

Training Epoch: 15 [450/13482]	Loss: 1464.5565
Training Epoch: 15 [500/13482]	Loss: 1480.7336
Training Epoch: 15 [550/13482]	Loss: 1485.9802
Training Epoch: 15 [600/13482]	Loss: 1480.5686
Training Epoch: 15 [650/13482]	Loss: 1493.5890
Training Epoch: 15 [700/13482]	Loss: 1521.6321
Training Epoch: 15 [750/13482]	Loss: 1472.7831
Training Epoch: 15 [800/13482]	Loss: 1483.4192
Training Epoch: 15 [850/13482]	Loss: 1455.8435
Training Epoch: 15 [900/13482]	Loss: 1448.0127
Training Epoch: 15 [950/13482]	Loss: 1492.8282
Training Epoch: 15 [1000/13482]	Loss: 1504.2201
Training Epoch: 15 [1050/13482]	Loss: 1442.8646
Training Epoch: 15 [1100/13482]	Loss: 1485.0308
Training Epoch: 15 [1150/13482]	Loss: 1477.8347
Training Epoch: 15 [1200/13482]	Loss: 1479.4545
Training Epoch: 15 [1250/13482]	Loss: 1456.2241
Training Epoch: 15 [1300/13482]	Loss: 1464.2042
Training Epoch: 15 [1350/13482]	Loss: 1440.9965
Training Epoch: 15 [1400/13482]	Loss: 1452.5767
Training Epoch: 15 [1450/13482]	Loss: 1438.2526
Tra

Training Epoch: 15 [9000/13482]	Loss: 1471.3302
Training Epoch: 15 [9050/13482]	Loss: 1465.8549
Training Epoch: 15 [9100/13482]	Loss: 1431.7549
Training Epoch: 15 [9150/13482]	Loss: 1473.5150
Training Epoch: 15 [9200/13482]	Loss: 1524.1969
Training Epoch: 15 [9250/13482]	Loss: 1405.6431
Training Epoch: 15 [9300/13482]	Loss: 1479.5132
Training Epoch: 15 [9350/13482]	Loss: 1438.6783
Training Epoch: 15 [9400/13482]	Loss: 1395.0214
Training Epoch: 15 [9450/13482]	Loss: 1471.7256
Training Epoch: 15 [9500/13482]	Loss: 1440.6649
Training Epoch: 15 [9550/13482]	Loss: 1441.5099
Training Epoch: 15 [9600/13482]	Loss: 1451.0543
Training Epoch: 15 [9650/13482]	Loss: 1430.1674
Training Epoch: 15 [9700/13482]	Loss: 1437.4922
Training Epoch: 15 [9750/13482]	Loss: 1429.6451
Training Epoch: 15 [9800/13482]	Loss: 1406.6462
Training Epoch: 15 [9850/13482]	Loss: 1403.5936
Training Epoch: 15 [9900/13482]	Loss: 1473.9526
Training Epoch: 15 [9950/13482]	Loss: 1442.5228
Training Epoch: 15 [10000/13482]	Loss: 1

Training Epoch: 16 [3950/13482]	Loss: 1395.6584
Training Epoch: 16 [4000/13482]	Loss: 1386.3258
Training Epoch: 16 [4050/13482]	Loss: 1435.9570
Training Epoch: 16 [4100/13482]	Loss: 1484.1295
Training Epoch: 16 [4150/13482]	Loss: 1412.6451
Training Epoch: 16 [4200/13482]	Loss: 1413.6744
Training Epoch: 16 [4250/13482]	Loss: 1437.5920
Training Epoch: 16 [4300/13482]	Loss: 1429.4039
Training Epoch: 16 [4350/13482]	Loss: 1406.0792
Training Epoch: 16 [4400/13482]	Loss: 1498.0541
Training Epoch: 16 [4450/13482]	Loss: 1429.8444
Training Epoch: 16 [4500/13482]	Loss: 1452.5579
Training Epoch: 16 [4550/13482]	Loss: 1386.7744
Training Epoch: 16 [4600/13482]	Loss: 1346.2751
Training Epoch: 16 [4650/13482]	Loss: 1446.9124
Training Epoch: 16 [4700/13482]	Loss: 1397.5525
Training Epoch: 16 [4750/13482]	Loss: 1361.9817
Training Epoch: 16 [4800/13482]	Loss: 1434.5605
Training Epoch: 16 [4850/13482]	Loss: 1444.2896
Training Epoch: 16 [4900/13482]	Loss: 1440.0625
Training Epoch: 16 [4950/13482]	Loss: 14